# Db2 JSON Function Overview
Updated: 2019-10-03

## Db2 JSON Functions
Db2 Version 11.1 Fix pack 4 introduced a subset of the JSON SQL functions defined by ISO and that set is shown in the table below.

| Function | Description |
|:---------|:------------|
| `BSON_TO_JSON` | Convert BSON formatted document into JSON strings 
| `JSON_TO_BSON` | Convert JSON strings into a BSON document format 
| `JSON_ARRAY` | Creates a JSON array from input key value pairs
| `JSON_OBJECT` | Creates a JSON object from input key value pairs 
| `JSON_VALUE` | Extract an SQL scalar value from a JSON object 
| `JSON_QUERY` | Extract a JSON object from a JSON object 
| `JSON_TABLE` | Creates a SQL table from a JSON object 
| `JSON_EXISTS` | Determines whether a JSON object contains the desired JSON value

These functions are all part of the SYSIBM schema, so a user does not require permissions in order to use them for development or general usage. The functions can be categorized into three broad categories:
#### Conversion functions
The `BSON_TO_JSON` and `JSON_TO_BSON` functions are used to convert JSON character data into the binary BSON format and vice-versa. Conversion functions are optional and are discussed in the section below. These functions are not actually part of the ISO specifications and are provided simply for your convenience.
#### Retrieval functions
The `JSON_VALUE` and `JSON_QUERY` functions are used to retrieve portions of a document as SQL or JSON scalar values, while `JSON_TABLE` can be used to format JSON documents into a table of rows and columns. The `JSON_EXISTS` function can be used in conjunction with the retrieval functions to check for the existence of a field.
#### Publishing Routines
The `JSON_ARRAY` and `JSON_OBJECT` functions are used to create JSON objects from relational data.

### Common Db2 JSON Parameters
A majority of the Db2 ISO JSON functions depend on two parameters that are supplied at the beginning of a function. These parameters are: 
* JSON Expression
* JSON Path Expression

#### JSON Expression
The JSON expression refers to either a column name in a table where the JSON document is stored (either in JSON or BSON format), a JSON or BSON literal string, or a SQL variable containing a JSON or BSON string. 

The examples below illustrate these options.
* A column name within a Table
```
JSON_VALUE(CUSTOMER.JSON_DOC,…)
```
* Using a character string as the argument
```
JSON_VALUE('{"first":"Thomas","last":"Hronis":}',…)
```

* Using an SQL variable
```
CREATE VARIABLE EXPR VARCHAR(256) DEFAULT('{"first":"Thomas"}')
JSON_VALUE(EXPR,…)
```

The JSON expression can also include a modifier of `FORMAT JSON` or `FORMAT BSON`. The `FORMAT` clause is optional and by default the Db2 functions use the data type of the supplied value to determine how to interpret the contents. In the event that you need to override how the JSON field is interpreted, you must use the `FORMAT` option.

#### JSON Path Expression
A JSON path expression is used to navigate to individual values, objects, arrays, or allow for multiple matches within a JSON document. The JSON path expression is based on the syntax that is fully described in the notebook on  JSON Path Expressions. 
The following list gives a summary of how a path expression is created but the details of how the matches occur are documented in the next chapter.
* The top of any path expression is the anchor symbol (`$`)
* Traverse to specific objects at different levels by using the dot operator (`.`)
* Use square brackets `[]` to refer to items in an array with the first item starting at position zero (i.e. first element in an array is accessed as `arrayname[0]`)
* Use the backslash `\` as an escape character when key names include any of the JSON path characters `(.,*,$,[,])`
* Use the asterisk (`*`) to match any object at the current level
* Use the asterisk (`*`) to match all objects in an array or retrieve only the value fields from an object

The path expression can have an optional name represented by the `AS path-name` clause. The `AS` clause is included for compatibility with the ISO SQL standard but currently does not have any effect on the Db2 JSON functions.

## Sample JSON Functions
The following SQL demonstrates some of the JSON functions that are available in Db2. The other notebooks will go into more details of each one of these functions.

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

This statement will create a variable named customer which will be used for some of the examples.

In [ ]:
customer = {
    "customerid": 100000,
    "identity": 
      {
        "firstname": "Jacob",
        "lastname": "Hines",
        "birthdate": "1982-09-18"
      },
    "contact": 
      {
        "street": "Main Street North",
        "city": "Amherst",
        "state": "OH",
        "zipcode": "44001",
        "email": "Ja.Hines@yahii.com",
        "phone": "813-689-8309"
      },
    "payment": 
      {
        "card_type": "MCCD",
        "card_no": "4742-3005-2829-9227"
      },
    "purchases": 
      [
        {
          "tx_date": "2018-02-14",
          "tx_no": 157972,
          "product_id": 1860,
          "product": "Ugliest Snow Blower",
          "quantity": 1,
          "item_cost": 51.86
        }
      ]
}

### JSON_EXISTS
Check to see if the customer made a __`purchase`__.

In [ ]:
%sql VALUES JSON_EXISTS(:customer,'$.purchases')

### JSON_VALUE
Retrieve the __`customerid`__ field.

In [ ]:
%sql VALUES JSON_VALUE(:customer,'$.customerid')

### JSON_QUERY
Retrieve the __`identity`__ structure.

In [ ]:
%sql -j VALUES JSON_QUERY(:customer,'$.identity')

### JSON_TABLE
Retrieve all of the personal information as a table.

In [ ]:
%%sql
WITH CUSTOMER(INFO) AS (VALUES :customer)
SELECT T.* FROM CUSTOMER, 
  JSON_TABLE(INFO, 'strict $'
    COLUMNS( 
      FIRST_NAME VARCHAR(20) PATH '$.identity.firstname',
      LAST_NAME  VARCHAR(20) PATH '$.identity.lastname',
      BIRTHDATE  DATE        PATH '$.identity.birthdate')
    ERROR ON ERROR) AS T;

### JSON_OBECT
Publish one record as a JSON object.

In [ ]:
%%sql -j
WITH CUSTOMER(CUSTNO, FIRSTNAME, LASTNAME, BIRTHDATE, INCOME) AS
  (
  VALUES
     (1, 'George', 'Baklarz', '1999-01-01', 50000)
  )
SELECT 
   JSON_OBJECT ( 
               KEY 'customer' VALUE JSON_OBJECT
                   ( 
                   KEY 'id' VALUE CUSTNO,
                   KEY 'name' VALUE JSON_OBJECT
                       (
                       KEY 'first' VALUE FIRSTNAME,
                       KEY 'last'  VALUE LASTNAME
                       ) FORMAT JSON,
                   KEY 'birthdate' VALUE BIRTHDATE,
                   KEY 'income'    VALUE INCOME
                   ) FORMAT JSON
               )
FROM CUSTOMER

### JSON_ARRAY
Publish one record as a JSON array object.

In [ ]:
%%sql -j
WITH CUSTOMERS(CUSTNO) AS
  (
  VALUES
     10, 20, 33, 55, 77
  )
VALUES 
   JSON_OBJECT ( 
               KEY 'customers' VALUE JSON_ARRAY (SELECT * FROM CUSTOMERS) FORMAT JSON
               ) 

### Summary
Db2 supports a number of the new ISO JSON functions directly in the database. Users can use this new syntax to store, query, and publish JSON data from within Db2.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]